In [18]:
import numpy as np


In [19]:
with open("./released/sents.train") as f:
    texts_train = f.read()
f.close()


In [20]:
words_train = texts_train.replace("\n", " ")[:-1].split(sep=" ")


In [21]:
prev_POS = "[START]"

# dict_train = {}
PennTreebankPOS = {"[START]": {}}
WordEmission = {}
for word_train in words_train:
    tag = word_train.split("/")[-1]
    word = word_train[: -(len(tag) + 1)]

    # # Multi part of speech
    # if not word in dict_train.keys():
    #     dict_train[word] = {tag}
    # else:
    #     if tag not in dict_train[word]:
    #         dict_train[word].add(tag)

    # Transition Probability Matrix
    if not tag in PennTreebankPOS.keys():  # Create new state
        PennTreebankPOS[tag] = {}
    if tag not in PennTreebankPOS[prev_POS].keys():  # a_ij from 0 to 1
        PennTreebankPOS[prev_POS][tag] = 1
    else:
        PennTreebankPOS[prev_POS][tag] += 1  # a_ij from x to x+1 (x > 0)

    # Observation Likelihood
    if not tag in WordEmission.keys():
        WordEmission[tag] = {word: 1}
    else:
        if not word in WordEmission[tag].keys():
            WordEmission[tag][word] = 1
        else:
            WordEmission[tag][word] += 1

    prev_POS = tag


In [33]:
def ProbScale(x: dict):
    out = x.copy()
    for i in out.keys():
        sum_i = sum(out[i].values())
        for j in out[i].keys():
            out[i][j] /= sum_i
    return out


In [27]:
POS_trans_mat = ProbScale(PennTreebankPOS)
obs_emi_mat = ProbScale(WordEmission)
